<a href="https://colab.research.google.com/github/pamelag/2014/blob/master/cover_letter_entity_relation_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3

In [ ]:
import spacy
from huggingface_hub import notebook_login
from docling.document_converter import DocumentConverter

In [ ]:
notebook_login()

In [ ]:
def load_spacy_embeddings():
  return spacy.load("en_core_web_sm")

In [ ]:
nlp = load_spacy_embeddings()

### Defining Cover Letter Document Structure

## Use Dockling to Extract Data from PDF

- Generating Doc Graph based on the Broad headings found in the document
- TODO - Need to keep the Paragraph Number for each para of Text

In [ ]:
from docling.document_converter import DocumentConverter

doc_paragraphs = []
source = "./sample_data/math_analysis_2.pdf"
converter = DocumentConverter()
result = converter.convert(source)
contents = result.document.texts

for text_item in contents:
  # print(text_item)
  para_ref = text_item.self_ref.replace("#/texts", "paragraph").replace("/", "_") + "_page_" + str(text_item.prov[0].page_no)
  content_type = text_item.label.TEXT
  page_no = text_item.prov[0].page_no
  bounding_box = text_item.prov[0].bbox
  original_text = text_item.orig
  text = text_item.text
  doc_paragraphs.append({"id": para_ref, "content_type": content_type, "page_no": page_no, "bounding_box": bounding_box, "original_text": original_text, "text": text, "sentences": {}})

print(doc_paragraphs)

[{'id': 'paragraph_0_page_1', 'content_type': <DocItemLabel.TEXT: 'text'>, 'page_no': 1, 'bounding_box': BoundingBox(l=0.0, t=0.0, r=0.0, b=0.0, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), 'original_text': '<unknown>', 'text': '<unknown>', 'sentences': {}}, {'id': 'paragraph_1_page_1', 'content_type': <DocItemLabel.TEXT: 'text'>, 'page_no': 1, 'bounding_box': BoundingBox(l=110.9000015258789, t=715.79296875, r=321.29998779296875, b=704.4990234375, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), 'original_text': 'By email to: duties@revenue.nsw.gov.au', 'text': 'By email to: duties@revenue.nsw.gov.au', 'sentences': {}}, {'id': 'paragraph_2_page_1', 'content_type': <DocItemLabel.TEXT: 'text'>, 'page_no': 1, 'bounding_box': BoundingBox(l=110.9000015258789, t=686.8729858398438, r=227.5800018310547, b=675.5789794921875, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), 'original_text': 'Private & Confidential', 'text': 'Private & Confidential', 'sentences': {}}, {'id': 'p

In [ ]:
from docling.document_converter import DocumentConverter

doc_paragraphs = []

doc_graph = {}

source = "./sample_data/math_analysis_2.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
contents = result.document.texts
children_text_list = []

for text_item in contents:
  if text_item.label == "section_header":
    children_text_list = []
    ref_no = text_item.self_ref.replace("#/", "").replace("/", "_") + "_" + str(text_item.prov[0].page_no) + "__" + text_item.orig
    doc_graph[ref_no] = children_text_list
  elif text_item.label == "text":
    children_text_list.append({"sentence": text_item.orig, "clauses": [], "rdf": {}})
  # print(text_item)
print(doc_graph)
print("----------------------------------------------------------")

{'texts_1_1__By email to: duties@revenue.nsw.gov.au': [], 'texts_2_1__Private & Confidential': [{'sentence': 'Chief Commissioner of State Revenue Revenue NSW The Lang Centre 132 Marsden Street PARAMATTA  NSW  2150', 'clauses': [], 'rdf': {}}, {'sentence': 'Our reference: Barry Diamond / Ilyas Elahi / Stephanie Wong', 'clauses': [], 'rdf': {}}, {'sentence': '10 August 2021', 'clauses': [], 'rdf': {}}, {'sentence': 'Dear Commissioner', 'clauses': [], 'rdf': {}}, {'sentence': 'Acquisition of 100% of the shares in Coca-Cola Amatil Limited Notification of transaction and lodgement under section 152 of the Duties Act 1997 (NSW)', 'clauses': [], 'rdf': {}}, {'sentence': 'We act on behalf of CCEP Australia Pty Ltd ( CCEPA ).', 'clauses': [], 'rdf': {}}, {'sentence': 'This letter concerns an acquisition by CCEPA of 100% of the issued share capital in Coca-Cola Amatil Limited ( CCA ) on 10 May 2021 ( Acquisition ), an entity that was listed on the Australian Securities Exchange ( ASX ).', 'claus

### Extract Clause Function

In [ ]:
import re
# def extract_chunks(clause, accumulator, clause_words):
#   # print("clause : ", clause)
#   # accumulator = []
#   # clause_words = []

#   def clause_word_exists(clause):
#     clause_word_list = [":", ",", "however", "hence", "under"]
#     for clause_word in clause_word_list:
#       # print("clause_word : ", clause_word)
#       if clause_word in clause:
#         clause_words.append(clause_word)
#         return True
#       return False

#   def split_clause(clause):
#     text_chunks_by_punctuations = []
#     clause_word_list = [":", ",","however", "hence", "under"]
#     for clause_word in clause_word_list:
#       if clause_word in clause:
#         text_chunks_by_punctuations = clause.split(clause_word)
#         break
#       else:
#         text_chunks_by_punctuations = [clause]
#     return text_chunks_by_punctuations

#   if clause_word_exists(clause):
#     splits = split_clause(clause)
#     if len(splits) > 1:
#       for split in splits:
#         extract_chunks(split, accumulator, clause_words)
#     else:
#       extract_chunks(splits[0], accumulator, clause_words)
#   else:
#     accumulator.append(clause)
#     return

def extract_chunks(clause):
  chunks = re.split('[,;:]', clause)
  return chunks

In [ ]:
text = "This letter concerns an acquisition by CCEPA of 100% of the issued share capital in Coca-Cola Amatil Limited (CCA) on 10 May 2021 (Acquisition), an entity that was listed on the Australian Securities Exchange (ASX)."
chunks = extract_chunks(text)

print(chunks)

['This letter concerns an acquisition by CCEPA of 100% of the issued share capital in Coca-Cola Amatil Limited (CCA) on 10 May 2021 (Acquisition)', ' an entity that was listed on the Australian Securities Exchange (ASX).']


In [ ]:
def extract_chunks_by_punctuations(clause):
  clause_splitters = []
  splits = []

  print(type(clause))
  print(clause)

  nlp = spacy.load("en_core_web_sm")
  doc = nlp(clause)

  for token in doc:
    if token.dep_ == "PUNCT":
      clause_splitters.append(token.text)

  for splitter in clause_splitters:
    splits = clause.split(splitter)

  return splits, clause_splitters

In [ ]:
extract_chunks_by_punctuations('''This letter concerns an acquisition by CCEPA of 100% of the issued share capital in Coca-Cola Amatil Limited (CCA) on 10 May 2021 (Acquisition), an entity that was listed on the Australian Securities Exchange (ASX).''')

<class 'str'>
This letter concerns an acquisition by CCEPA of 100% of the issued share capital in Coca-Cola Amatil Limited (CCA) on 10 May 2021 (Acquisition), an entity that was listed on the Australian Securities Exchange (ASX).


([], [])

### Doc Graph Construction

- Get the Noun chunks
- Get the Verbs
- Arrange them Sequentially
- Arrange them in Head - Relation - Tail format
- Create a Chain
- Each Item in a Chain should have the Text, Qualifier ( Noun Phrase), POS Tag ans Type (ORG/Date etc)

In [ ]:
def analyse_sentence(sentence):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(sentence)

  rdf_dict = {}
  noun_chunks = [chunk.text for chunk in doc.noun_chunks]
  verbs = [token.text for token in doc if token.pos_ == "VERB"]

  word_counter = 0
  root_index = 0

  head = ""
  relation = ""
  tail = ""

  head_relation_tail_list = []

  for token in doc:
    if token.pos_ == "VERB" or token.dep_ == "ROOT":
      head_tail_split = sentence.split(token.text)
      head = head_tail_split[0]
      relation = token.text
      tail = head_tail_split[1]

      head_relation_tail_list.append({"head": head, "relation": relation, "tail": tail})

  return head_relation_tail_list, noun_chunks, verbs

In [ ]:
def analyse_noun_chunks(sentence):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(sentence)

  chunk_root = ""
  subject = ""
  direct_object = ""

  sentence_graph = {}
  relation_entity_list = []

  sentence_graph["id"] = "artifact_1_1"

  for chunk in doc.sents:
      chunk_root = chunk.root.text

  for chunk in doc.noun_chunks:
    # print("chunk.text : ", chunk.text)
    # print("chunk.root.text : ", chunk.root.text)
    # print("chunk.root.dep_ ", chunk.root.dep_)
    # print("chunk.root.head.text ", chunk.root.head.text)
    # print("------------------------------------------------------")
    if chunk.root.dep_ in ["nsubj",  "csubj", "csubjpass"]:
        subject = chunk.text

    if chunk.root.dep_ == "nsubjpass":
      subject = chunk.text
      chunk_split = sentence.split(chunk.root.head.text)[1]
      sentence_graph["head_relation_tail"] =  {"head": subject, "relation": chunk.root.head.text, "tail": chunk_split}

    if chunk.root.dep_  == "dobj":
      direct_object = chunk.text
      chunk_split = sentence.split(chunk.root.head.text)[1]
      relation_entity_list.append({"relation": chunk.root.head.text, "tail": chunk.text})
      sentence_graph["sub_pred_obj"] =  {"head": subject, "relation": chunk_root, "tail": direct_object}

    if chunk.root.dep_  == "pobj" or chunk.root.dep_  == "iobj":
      relation_entity_list.append({"relation": chunk.root.head.text, "tail": chunk.text})

  sentence_graph["entities_relations"] = relation_entity_list

  return sentence_graph

In [ ]:
analyse_sentence("We have been asked to assist CCCEPA with compliance of its obligations under the Duties Act 1997 (NSW) (the Act ) in relation to the Acquisition.")

([{'head': 'We have been ',
   'relation': 'asked',
   'tail': ' to assist CCCEPA with compliance of its obligations under the Duties Act 1997 (NSW) (the Act ) in relation to the Acquisition.'},
  {'head': 'We have been asked to ',
   'relation': 'assist',
   'tail': ' CCCEPA with compliance of its obligations under the Duties Act 1997 (NSW) (the Act ) in relation to the Acquisition.'}],
 ['We',
  'CCCEPA',
  'compliance',
  'its obligations',
  'the Duties Act',
  '(NSW',
  'the Act',
  'relation',
  'the Acquisition'],
 ['asked', 'assist'])

In [ ]:
analyse_noun_chunks("We have been asked to assist CCCEPA with compliance of its obligations under the Duties Act 1997 (NSW) (the Act ) in relation to the Acquisition.")

{'id': 'artifact_1_1',
 'head_relation_tail': {'head': 'We',
  'relation': 'asked',
  'tail': ' to assist CCCEPA with compliance of its obligations under the Duties Act 1997 (NSW) (the Act ) in relation to the Acquisition.'},
 'sub_pred_obj': {'head': 'We', 'relation': 'asked', 'tail': 'CCCEPA'},
 'entities_relations': [{'relation': 'assist', 'tail': 'CCCEPA'},
  {'relation': 'with', 'tail': 'compliance'},
  {'relation': 'of', 'tail': 'its obligations'},
  {'relation': 'under', 'tail': 'the Duties Act'},
  {'relation': 'in', 'tail': 'relation'},
  {'relation': 'to', 'tail': 'the Acquisition'}]}

### Parsing Each sentence in a paragraph from the Doc Graph

In [ ]:
def parse_paragraph(paragraph_dict):
  sentence_list = []
  counter = 0
  paragraph_id = paragraph_dict["id"]
  text = paragraph_dict["original_text"]

  sentence_list = paragraph_dict["original_text"].split(".")
  if len(sentence_list) > 1:

    for sentence in sentence_list:
      text_chunks = []
      rdf_list = []
      counter += 1
      sentence_id = counter
      text_chunks = extract_chunks(sentence)

      for text_chunk in text_chunks:
        head, relation, tail = analyse_sentence(text_chunk)
        rdf_list.append({"head": head, "relation": relation, "tail": tail})


    sentence_list.append({"paragraph_id": paragraph_id, "sentence_id": sentence_id, "sentence": sentence, "chunks": text_chunks, "rdf_list": rdf_list})

  return sentence_list

In [ ]:
def generate_rdf(doc_paragraphs):
  for  paragraph_dict in doc_paragraphs:
    sentence_list = parse_paragraph(paragraph_dict)
    paragraph_dict["sentences"] = sentence_list

In [ ]:
generate_rdf(doc_paragraphs)

In [ ]:
doc_paragraphs

[{'id': 'paragraph_0_page_1',
  'content_type': <DocItemLabel.TEXT: 'text'>,
  'page_no': 1,
  'bounding_box': BoundingBox(l=0.0, t=0.0, r=0.0, b=0.0, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>),
  'original_text': '<unknown>',
  'text': '<unknown>',
  'sentences': ['<unknown>']},
 {'id': 'paragraph_1_page_1',
  'content_type': <DocItemLabel.TEXT: 'text'>,
  'page_no': 1,
  'bounding_box': BoundingBox(l=110.9000015258789, t=715.79296875, r=321.29998779296875, b=704.4990234375, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>),
  'original_text': 'By email to: duties@revenue.nsw.gov.au',
  'text': 'By email to: duties@revenue.nsw.gov.au',
  'sentences': ['By email to: duties@revenue',
   'nsw',
   'gov',
   'au',
   {'paragraph_id': 'paragraph_1_page_1',
    'sentence_id': 4,
    'sentence': 'au',
    'chunks': ['au'],
    'rdf_list': [{'head': '', 'relation': '', 'tail': ''}]}]},
 {'id': 'paragraph_2_page_1',
  'content_type': <DocItemLabel.TEXT: 'text'>,
  'page_no': 1,
  

## Experimental Snippets

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("whereby since however hence under")
# doc = nlp('''We have been asked to assist CCCEPA with compliance of its obligations under the Duties Act 1997 (NSW) (the Act) in relation to the Acquisition''')

for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_)

whereby SCONJ WRB advmod
since SCONJ IN prep
however ADV RB pcomp
hence ADV RB pcomp
under ADP IN ROOT


In [ ]:
def extract_chunks(clause, accumulator, clause_words):
  # print("clause : ", clause)
  accumulator = []
  clause_words = []

  def clause_word_exists(clause):
    clause_word_list = [":", ",", "however", "hence", "under"]
    for clause_word in clause_word_list:
      # print("clause_word : ", clause_word)
      if clause_word in clause:
        clause_words.append(clause_word)
        return True
      return False

  def split_clause(clause):
    text_chunks_by_punctuations = []
    clause_word_list = [",","however", "hence", "under"]
    for clause_word in clause_word_list:
      if clause_word in clause:
        text_chunks_by_punctuations = clause.split(clause_word)
        break
      else:
        text_chunks_by_punctuations = [clause]
    return text_chunks_by_punctuations

  if clause_word_exists(clause):
    splits = split_clause(clause)
    if len(splits) > 1:
      for split in splits:
        extract_chunks(split, accumulator, clause_words)
    else:
      extract_chunks(splits[0], accumulator, clause_words)
  else:
    accumulator.append(clause)
    return accumulator, clause_words

In [ ]:
# # pip install -U spacy
# # python -m spacy download en_core_web_sm
# import spacy

# # Load English tokenizer, tagger, parser and NER
# nlp = spacy.load("en_core_web_sm")

# # Process whole documents
# # text = ('''We have been asked to assist CCCEPA with compliance of its obligations under the Duties Act 1997 (NSW) (the Act) in relation to the Acquisition. On behalf of CCEPA, we request the Chief
# # Commissioner of State Revenue to review the information provided in this letter and issue an assessment in relation to the Acquisition.''')

# text = ('''On 4 November 2020, Coca-Cola European Partners plc and the independent shareholders of CCA entered into a Scheme Implementation Deed whereby it was agreed that Coca-Cola European Partners plc would acquire all of the issued shares held by the independent shareholders''')

# doc = nlp(text)

# # Analyze syntax
# print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
# print("Verbs:", [token.text for token in doc if token.pos_ == "VERB"])

# # Find named entities, phrases and concepts
# for entity in doc.ents:
#     print(entity.text, entity.label_)

Noun phrases: ['4 November', 'Coca-Cola European Partners plc', 'the independent shareholders', 'CCA', 'a Scheme Implementation Deed', 'it', 'Coca-Cola European Partners plc', 'all', 'the issued shares', 'the independent shareholders']
Verbs: ['entered', 'agreed', 'acquire', 'issued', 'held']
4 November 2020 DATE
Coca-Cola European Partners plc ORG
CCA ORG
Coca-Cola European Partners plc ORG


In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp('''On 4 November 2020, Coca-Cola European Partners plc and the independent shareholders of CCA entered into a Scheme Implementation Deed whereby it was agreed that Coca-Cola European Partners plc would acquire all of the issued shares held by the independent shareholders''')
for chunk in doc.noun_chunks:
  print("chunk.text : ", chunk.text)
  print("chunk.root.text : ", chunk.root.text)
  print("chunk.root.dep_ ", chunk.root.dep_)
  print("chunk.root.head.text ", chunk.root.head.text)

chunk.text :  4 November
chunk.root.text :  November
chunk.root.dep_  pobj
chunk.root.head.text  On
chunk.text :  Coca-Cola European Partners plc
chunk.root.text :  plc
chunk.root.dep_  nsubj
chunk.root.head.text  entered
chunk.text :  the independent shareholders
chunk.root.text :  shareholders
chunk.root.dep_  conj
chunk.root.head.text  plc
chunk.text :  CCA
chunk.root.text :  CCA
chunk.root.dep_  pobj
chunk.root.head.text  of
chunk.text :  a Scheme Implementation Deed
chunk.root.text :  Deed
chunk.root.dep_  pobj
chunk.root.head.text  into
chunk.text :  it
chunk.root.text :  it
chunk.root.dep_  nsubjpass
chunk.root.head.text  agreed
chunk.text :  Coca-Cola European Partners plc
chunk.root.text :  plc
chunk.root.dep_  nsubj
chunk.root.head.text  acquire
chunk.text :  all
chunk.root.text :  all
chunk.root.dep_  dobj
chunk.root.head.text  acquire
chunk.text :  the issued shares
chunk.root.text :  shares
chunk.root.dep_  pobj
chunk.root.head.text  of
chunk.text :  the independent shareh

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp('''On 4 November 2020, Coca-Cola European Partners plc and the independent shareholders of CCA entered into a Scheme Implementation Deed whereby it was agreed that Coca-Cola European Partners plc would acquire all of the issued shares held by the independent shareholders''')
for chunk in doc.sents:
  print("chunk.text : ", chunk.text)
  print("chunk.root.text : ", chunk.root.text)
  print("chunk.root.dep_ ", chunk.root.dep_)
  print("chunk.root.head.text ", chunk.root.head.text)

chunk.text :  On 4 November 2020, Coca-Cola European Partners plc and the independent shareholders of CCA entered into a Scheme Implementation Deed whereby it was agreed that Coca-Cola European Partners plc would acquire all of the issued shares held by the independent shareholders
chunk.root.text :  entered
chunk.root.dep_  ROOT
chunk.root.head.text  entered


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp('''On 4 November 2020, Coca-Cola European Partners plc and the independent shareholders of CCA entered into a Scheme Implementation Deed whereby it was agreed that Coca-Cola European Partners plc would acquire all of the issued shares held by the independent shareholders''')

for token in doc:
    print(token.text, token.pos_, token.dep_)

On ADP prep
4 NUM nummod
November PROPN pobj
2020 NUM nummod
, PUNCT punct
Coca PROPN compound
- PUNCT punct
Cola PROPN compound
European PROPN compound
Partners PROPN compound
plc PROPN nsubj
and CCONJ cc
the DET det
independent ADJ amod
shareholders NOUN conj
of ADP prep
CCA PROPN pobj
entered VERB ROOT
into ADP prep
a DET det
Scheme PROPN compound
Implementation PROPN compound
Deed PROPN pobj
whereby SCONJ advmod
it PRON nsubjpass
was AUX auxpass
agreed VERB advcl
that SCONJ mark
Coca PROPN compound
- PUNCT punct
Cola PROPN compound
European PROPN compound
Partners PROPN compound
plc PROPN nsubj
would AUX aux
acquire VERB ccomp
all PRON dobj
of ADP prep
the DET det
issued VERB amod
shares NOUN pobj
held VERB acl
by ADP agent
the DET det
independent ADJ amod
shareholders NOUN pobj


In [ ]:
doc_1 = nlp('''We have been asked to assist CCCEPA with compliance of its obligations under the Duties Act 1997 (NSW) (the Act) in relation to the Acquisition.''')
# doc_2 = nlp('''an entity that was listed on the Australian Securities Exchange (ASX).''')

sentence = '''We have been asked to assist CCCEPA with compliance of its obligations under the Duties Act 1997 (NSW) (the Act) in relation to the Acquisition.'''

chunk_root = ""
subject = ""
direct_object = ""

sentence_graph = {}
relation_entity_list = []

sentence_graph["id"] = "artifact_1_1"

for chunk in doc_1.sents:
    chunk_root = chunk.root.text

for chunk in doc_1.noun_chunks:
  print("chunk.text : ", chunk.text)
  print("chunk.root.text : ", chunk.root.text)
  print("chunk.root.dep_ ", chunk.root.dep_)
  print("chunk.root.head.text ", chunk.root.head.text)
  print("------------------------------------------------------")
  if chunk.root.dep_ in ["nsubj",  "csubj", "csubjpass"]:
      subject = chunk.text

  if chunk.root.dep_ == "nsubjpass":
    subject = chunk.text
    chunk_split = sentence.split(chunk.root.head.text)[1]
    sentence_graph["head_relation_tail"] =  {"head": subject, "relation": chunk.root.head.text, "tail": chunk_split}

  if chunk.root.dep_  == "dobj":
    direct_object = chunk.text
    chunk_split = sentence.split(chunk.root.head.text)[1]
    relation_entity_list.append({"relation": chunk.root.head.text, "tail": chunk.text})
    sentence_graph["sub_pred_obj"] =  {"head": subject, "relation": chunk_root, "tail": direct_object}

  if chunk.root.dep_  == "pobj" or chunk.root.dep_  == "iobj":

    relation_entity_list.append({"relation": chunk.root.head.text, "tail": chunk.text})




sentence_graph["entities_relations"] = relation_entity_list

print(sentence_graph)


chunk.text :  We
chunk.root.text :  We
chunk.root.dep_  nsubjpass
chunk.root.head.text  asked
------------------------------------------------------
chunk.text :  CCCEPA
chunk.root.text :  CCCEPA
chunk.root.dep_  dobj
chunk.root.head.text  assist
------------------------------------------------------
chunk.text :  compliance
chunk.root.text :  compliance
chunk.root.dep_  pobj
chunk.root.head.text  with
------------------------------------------------------
chunk.text :  its obligations
chunk.root.text :  obligations
chunk.root.dep_  pobj
chunk.root.head.text  of
------------------------------------------------------
chunk.text :  the Duties Act
chunk.root.text :  Act
chunk.root.dep_  pobj
chunk.root.head.text  under
------------------------------------------------------
chunk.text :  (NSW
chunk.root.text :  NSW
chunk.root.dep_  appos
chunk.root.head.text  Act
------------------------------------------------------
chunk.text :  the Act
chunk.root.text :  Act
chunk.root.dep_  appos
chunk

In [ ]:
from docling.document_converter import DocumentConverter

source = "./sample_data/algebra_2.pdf"
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())

<!-- image -->

## Acquisition Statement: Acquisition of an Interest in a Public Landholder

This statement must be made by the person who has made the relevant acquisition in a private landholder on or after 24 June 2020. The statement also facilitates;

-  Commonwealth Reporting Requirements

to collect and report to the Australian Tax Office (ATO), information on relevant acquisitions of land in NSW

## Note:

-  Under the Taxation Administration Act 1996 , you are required to provide all relevant information to enable duty to be assessed on a document or transaction. It is an offence to provide false or misleading information
-  Each person who makes a relevant acquisition must complete a separate acquisition statement
-  Read the notes at the back of this form for an explanation of the terms used in this statement
-  If you need more room, complete your answers on a separate sheet and attach to this form when lodging
-  For an exempt acquisition, use form ODA 047
-  For an 